Kévin TANG

Yann VINCENT

In [1]:
import networkx
import numpy
import matplotlib
import torch
import sklearn
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.13.0+cpu.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 995.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=495091 sha256=0ca0d16b2d49c461f4163ca7314e6b28a65b4b44f827485a7d65dc49b0e1387b
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.0/210.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-sparse: filename=torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl size=1035675 sha256=d9255ffdc39c370f37133b659c8361e50aa4827fcc67c27d99861b7f8eb78d77
  Stored in directory: /root/.cache/pip/wheels/c9/dd/0f/a6a16f9f3b0236733d257b4b4ea91b548b984a341ed3b8f38c
Successfully built to

# Partie 1

Nous commençons par voir comment PyG stocke les graphes en tenseurs. Ensuite, nous verrons un petit exemple d’utilisation du package OGB.

## 1. PyG Dataset et Data
PyG a deux classes pour stocker et transformer les graphes en Tenseurs. La première est `torch_geometric.datasets` qui contient une variété d’ensemble de données graphes. La deuxième classe `torch_geometric.data` qui permet de manipuler des graphes en tant que tenseurs.


### - PyG Dataset

Exécuter la cellule ci-dessous pour télécharger l'ensemble de données de classification de graphes ENZYMES. Ce dataset contient 600 graphes.

In [2]:
from torch_geometric.datasets import TUDataset

root = './enzymes'
name = 'ENZYMES'

# Téléchargement l'ensemble de données ENZYMES
pyg_dataset= TUDataset('./enzymes', 'ENZYMES')

# Cet Ensemble de données contient 600 graphes
print(pyg_dataset)

Extracting enzymes/ENZYMES/ENZYMES.zip
Processing...


ENZYMES(600)


Done!


###  Quel est le nombre de classes et le nombre de features dans le dataset ENZYMES?


In [3]:
##############################Votre Code ici##############################
num_classes = pyg_dataset.num_classes
num_features = pyg_dataset.num_node_features
##########################################################################
print("{} dataset a {} classes".format(name, num_classes))
print("{} dataset a {} features".format(name, num_features))

ENZYMES dataset a 6 classes
ENZYMES dataset a 3 features


### - PyG Data

In [4]:
#  pyg_dataset est un dataset de classification de graphes
graph_0 = pyg_dataset[0]
print(graph_0)

Data(edge_index=[2, 168], x=[37, 3], y=[1])


### Quelle est la classe (label) du graphe (index 50 dans l'ensemble de données ENZYMES) ?

In [5]:
############# Votre code ici ############
idx = 50
label = pyg_dataset[50].y.item()
#########################################
print('Le graphe {} a le label {}'.format(idx, label))

Le graphe 50 a le label 5


### Quelle est le nombre d'arêtes du graphe d'index 100?

In [6]:
 ############# Votre code ici ############
  ## Remarque:
  ## 1. Vous ne pouvez pas retourner data.num_edges directement
  ## 2. Le graphe est non orienté
idx = 100
edge_index = pyg_dataset[idx].edge_index
num_edges = edge_index.size(1) // 2
 #########################################
print('Le graphe {} a {} arêtes'.format(idx, num_edges))

Le graphe 100 a 88 arêtes


## 2. Open Graph Benchmark (OGB)

L’OGB est une collection des ensembles de données réels  de grande échelle utilisés pour l’apprentissage automatique sur les graphes. Ces ensembles de données peuvent être téléchargés, traités, est divisés (entrainement, validation et test) automatiquement en utilisant l’OGB Data Loader. L’OGB permet également d’évaluer les performances des différents modèles d’apprentissage d’une manière unifiée.  L’OGB prend également en charge les ensembles de données et les données PyG.

Nous allons jeter un coup d’œil sur l’ensemble de données `ogbn-arxiv`.

- Exécuter la cellule ci-dessous pour charger l'ensemble de données et le transformer en tenseur.

In [7]:
!pip install ogb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7026 sha256=448e384fa078230ffdf2c25e3fc7863a108c4796dbef0d0d189f78ec5e30eacb
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463cc03cf7379889266
Successfully built littleutils


In [8]:
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset

dataset_name = 'ogbn-arxiv'
# charger le dataset et le transformer en sparse tensor
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
print('Le {} dataset a {} graphe'.format(dataset_name, len(dataset)))

# Extract the graph
data = dataset[0]
print(data)

Downloaded 0.08 GB: 100%|██████████| 81/81 [00:09<00:00,  8.32it/s]


Extracting dataset/arxiv.zip


Processing...


Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00, 1629.49it/s]


Converting graphs into PyG objects...


100%|██████████| 1/1 [00:00<00:00, 2947.51it/s]

Saving...



Done!


Le ogbn-arxiv dataset a 1 graphe
Data(num_nodes=169343, x=[169343, 128], node_year=[169343, 1], y=[169343, 1], adj_t=[169343, 169343, nnz=1166243])


### Quel est le nombre de features dans le graphe ogbn-arxiv ?

In [9]:
################ Votre code ici ##########

num_features =  data.num_node_features

############################################
print('le graphe a {} features'.format(num_features))

le graphe a 128 features


# Partie 2 : GNN pour la classification de noeuds

Dans cette partie, nous allons construire notre premier réseau de neurones de graphes (GNN) en utilisant PyG et nous allons l'appliquer à la tache de prédiction de propriétés de nœuds (classification de nœuds). Notre modèle sera basé sur l’opérateur de convolution de graphes GCN (Kipf et al.2017). On ne vous demande pas d’implémenter cet opérateur. Vous devez utiliser directement la couche `GCNConv` de PyG.

(Kipf et al.2017) : https://arxiv.org/pdf/1609.02907.pdf


## Setup

In [10]:
import torch
import torch.nn.functional as F
print(torch.__version__)

# la couche GCNConv de PyG
from torch_geometric.nn import GCNConv

import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

2.1.0+cu121


## Chargement et prétraitement de données

In [11]:
dataset_name = 'ogbn-arxiv'
dataset = PygNodePropPredDataset(name=dataset_name,
                                 transform=T.ToSparseTensor())
data = dataset[0]

# Transformer la matrice d'adjacence en matrice Symétrique
data.adj_t = data.adj_t.to_symmetric()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Si vous utilisez un GPU, le device devrait etre cuda
print('Device: {}'.format(device))

# Diviser les données
data = data.to(device)
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

Device: cpu


## Le modèle GCN

Nous allons maintenant implémenter le modèle suivant.

![image.png](attachment:image.png)

In [12]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, return_embeds=False):
        # à faire : Implémenter cette fonction qui initialise self.convs, self.bns et self.softmax

        super(GCN, self).__init__()



        ################# Your code here ###########################################
        ## Remarques :
        ## 1. Vous devez utiliser torch.nn.ModuleList pour self.convs et self.bns
        ## 2. self.convs a num_layers de couches GCNConv
        ## 3. self.bns a (num_layers - 1) de couches BatchNorm1d
        ## 4. Utilisez torch.nn.LogSoftmax pour self.softmax
        ## 5. Les paramètres que vous pouvez passer à la couche  GCNConv sont 'in_channels' et
        ## 'out_channels'. Lien vers la documentation pour plus d'information :
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv
        ## 6. le seul paramètre que vous devez passer à BatchNorm1d est  'num_features'
        ## Lien vers la documentation pour plus d'information :
        ## https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html


        # Une liste des couches GCNConv  : Vore code Ici
        self.convs = torch.nn.ModuleList(
            [GCNConv(in_channels=input_dim, out_channels=hidden_dim)] +
            [GCNConv(in_channels=hidden_dim, out_channels=hidden_dim) for i in range(num_layers-2)] +
            [GCNConv(in_channels=hidden_dim, out_channels=output_dim)]
        )

        # Une liste de couches 1D de batch normalization : Votre Code Ici
        self.bns = torch.nn.ModuleList(
            [torch.nn.BatchNorm1d(num_features=hidden_dim) for i in range(num_layers-1)]
        )


        # la couche log softmax layer : Votre Code Ici
        self.softmax = torch.nn.LogSoftmax(dim=1)

       ########################################################################################

        # Probabilité d'un élement d'etre mis à 0
        self.dropout = dropout

        # Sauter la couche de  classification et retourner les embeddings
        self.return_embeds = return_embeds

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        # à faire : Implementer la fonction forward
        # Entrées : feature tensor x, edge_index_tensor adj_t
        # Retour : Tensor Voir la figure

        for conv, bn in zip(self.convs[:-1], self.bns):
            x1 = F.relu(bn(conv(x, adj_t)))
            if self.training:
                x1 = F.dropout(x1, p=self.dropout)
            x = x1
        x = self.convs[-1](x, adj_t)
        out = x if self.return_embeds else self.softmax(x)

        return out

In [13]:
def train(model, data, train_idx, optimizer, loss_fn):
    # à faire : implémenter la fonction train qui entraine le modèle
    # en utilisant les argument en entrée : optimizer et loss_fn
    model.train()
    loss = 0

    ##################################### Vore coode ici #####################################
    ## Remarques :
    ## 1. N'oubliez pas mettre les gradients à zero : zero_grad()
    ## 2. Passez data au modèle
    ## 3. Découpez (Tranchez) la sortie du modèle et de les étiquettes data.y par train_idx
    ## 4. Passez l'ouput et les étiquettes tranchés à loss_fn

    optimizer.zero_grad()
    output = model(data.x, data.adj_t)
    output = output[train_idx]
    loss = loss_fn(output, data.y[train_idx].squeeze())


    ##########################################################################################

    loss.backward()
    optimizer.step()

    return loss.item()

In [14]:
# La fonction Test
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    # à faire : Implémenter cette fonction qui test votre modèle
    # en utilisant les arguments split_idx and evaluator.
    model.eval()

    # L'output du model de tous les élements de data
    #out = None

    ############# Votre code ici #########################################
    ## Remarque: (Une ligne de code)
    ## 1. Aucun découpage des données par index ici

    out = model(data.x, data.adj_t)

    ##############################################################################

    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [15]:
# Ne changez pas les valeurs des ces arguments
args = {
    'device': device,
    'num_layers': 3,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.01,
    'epochs': 100,
}
args

{'device': 'cpu',
 'num_layers': 3,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.01,
 'epochs': 100}

In [16]:
model = GCN(data.num_features, args['hidden_dim'],
            dataset.num_classes, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbn-arxiv')

In [17]:
import copy

# reset the parameters to initial random value
model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = F.nll_loss

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  loss = train(model, data, train_idx, optimizer, loss_fn)
  result = test(model, data, split_idx, evaluator)
  train_acc, valid_acc, test_acc = result
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

/usr/local/lib/python3.10/dist-packages/torch_sparse/tensor.py:574: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(rowptr, col, value, self.sizes())


Epoch: 01, Loss: 4.1659, Train: 22.46%, Valid: 27.01% Test: 24.21%
Epoch: 02, Loss: 2.3752, Train: 24.07%, Valid: 21.14% Test: 26.22%
Epoch: 03, Loss: 1.9685, Train: 26.47%, Valid: 29.31% Test: 28.53%
Epoch: 04, Loss: 1.8016, Train: 32.51%, Valid: 30.99% Test: 28.92%
Epoch: 05, Loss: 1.6953, Train: 33.50%, Valid: 34.15% Test: 32.33%
Epoch: 06, Loss: 1.5994, Train: 29.68%, Valid: 27.13% Test: 29.56%
Epoch: 07, Loss: 1.5262, Train: 31.49%, Valid: 28.04% Test: 30.64%
Epoch: 08, Loss: 1.4805, Train: 34.90%, Valid: 33.96% Test: 35.37%
Epoch: 09, Loss: 1.4307, Train: 38.26%, Valid: 39.54% Test: 39.45%
Epoch: 10, Loss: 1.3853, Train: 41.53%, Valid: 44.27% Test: 43.87%
Epoch: 11, Loss: 1.3508, Train: 44.71%, Valid: 47.85% Test: 48.42%
Epoch: 12, Loss: 1.3169, Train: 46.15%, Valid: 47.80% Test: 49.96%
Epoch: 13, Loss: 1.2949, Train: 47.08%, Valid: 46.60% Test: 49.17%
Epoch: 14, Loss: 1.2712, Train: 49.12%, Valid: 47.81% Test: 49.81%
Epoch: 15, Loss: 1.2521, Train: 52.07%, Valid: 50.96% Test: 52

In [18]:
best_result = test(best_model, data, split_idx, evaluator)
train_acc, valid_acc, test_acc = best_result
print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Best model: Train: 73.66%, Valid: 71.98% Test: 70.97%


# Partie 3 : GNN pour la classification de graphes

Dans cette partie, nous allons créer un réseau de neurones de graphes GNN pour la prédiction des propriétés de graphes (classification de graphes)

## Chargement et prétraitement des données

In [19]:
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.data import DataLoader
from tqdm.notebook import tqdm

# Chargement du dataset
dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: {}'.format(device))

split_idx = dataset.get_idx_split()

# Afficher le type de la tâche (ça devrait afficher  classification binaire)
print('Task type: {}'.format(dataset.task_type))

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:03<00:00,  1.09s/it]
Processing...


Extracting dataset/hiv.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 41127/41127 [00:00<00:00, 84155.53it/s]


Converting graphs into PyG objects...


100%|██████████| 41127/41127 [00:01<00:00, 21367.01it/s]


Saving...
Device: cpu
Task type: binary classification


Done!


In [20]:
# chargement des données
# On va entrainer le modèle de classification de graphes sur des batchs de 32 graphes
# Mélanger (Shuffle) l'ordre des graphes dans l'ensemble d'entrainement
train_loader = DataLoader(dataset[split_idx["train"]], batch_size=32, shuffle=True, num_workers=0)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(dataset[split_idx["test"]], batch_size=32, shuffle=False, num_workers=0)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [21]:
# Ne changez pas ces valeurs
args = {
    'device': device,
    'num_layers': 5,
    'hidden_dim': 256,
    'dropout': 0.5,
    'lr': 0.001,
    'epochs': 30,
}
args

{'device': 'cpu',
 'num_layers': 5,
 'hidden_dim': 256,
 'dropout': 0.5,
 'lr': 0.001,
 'epochs': 30}

## Le modèle de  classification de graphes

Nous allons maintenant impélementer notre GCN pour la prédiction des propriétés de graphes (classification de graphes)

Nous allons réutiliser le modèle précèdent pour générer les plongements des nœuds et utiliser un Pooling global sur les nœuds pour prédire les propriétés du graphe en entier.

In [22]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool

### GCN to predict graph property
class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout):
        super(GCN_Graph, self).__init__()

        # Load encoders for Atoms in molecule graphs
        self.node_encoder = AtomEncoder(hidden_dim)

        # Node embedding model
        # Note that the input_dim and output_dim are set to hidden_dim
        self.gnn_node = GCN(9, hidden_dim,
            hidden_dim, num_layers, dropout, return_embeds=False)
        # Avant :
        # self.gnn_node = GCN(hidden_dim, hidden_dim,
        #   hidden_dim, num_layers, dropout, return_embeds=False)


        ############# Your code here ############
        ## Note:
        ## 1. Initialisez  self.pool par global mean pooling layer
        ## Lien vers la documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#global-pooling-layers
        ## (une ligne de code )

        #self.pool = None
        self.pool = global_mean_pool
        #########################################

        # Output layer
        self.linear = torch.nn.Linear(hidden_dim, output_dim)


    def reset_parameters(self):
      self.gnn_node.reset_parameters()
      self.linear.reset_parameters()

    def forward(self, batched_data):
        # Input :  tensor batched_data,
        # Output : tensor for each graph.
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch
        embed = self.node_encoder(x)
        #out = None

        ############# Votre code ici ###########################################
        ## Note:
        ## 1. Contruire les plongements de noeuds en utilisant le GCN précédent
        ## 2. Utiliser global pooling layer pour constuire les features du graphe en entier
        ## Lien vers la documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#global-pooling-layers
        ## 3. Utilisez le couche linear layer  pour prédire la propriété du graphe

        out = self.gnn_node(x.float(), edge_index)
        out = self.pool(out, batch)  # Appliquer le pooling global
        out = self.linear(out)  # Couche de sortie linéaire

        ########################################################

        return out

In [23]:
def train(model, device, data_loader, optimizer, loss_fn):

    model.train()
    loss = 0

    for step, batch in enumerate(tqdm(data_loader, desc="Iteration")):
      batch = batch.to(device)

      if batch.x.shape[0] == 1 or batch.batch[-1] == 0:
          pass
      else:
        ## ignorer nan targets (unlabeled) lors du calcul du training loss.
        is_labeled = batch.y == batch.y

        ############# Votre code ici ############
        ## Remarques:
        ## 1. N'oubliez pas mettre les gradients à zero : zero_grad()
        ## 2. Passez data au modèle
        ## 3. Utilisez le masque 'is_labeled' pour filtrer outut et labels
        ## 4. Changez le type du label à float
        ## 5. Passez l'ouput et les étiquettes tranchés à loss_fn

        optimizer.zero_grad()
        output = model(batch)
        output = output[is_labeled]
        target = batch.y[is_labeled].float()
        loss = loss_fn(output, target)



        #########################################

        loss.backward()
        optimizer.step()

    return loss.item()

In [24]:
# La fonction d'évaluation
def eval(model, device, loader, evaluator):
    model.eval()
    y_true = []
    y_pred = []

    for step, batch in enumerate(tqdm(loader, desc="Iteration")):
        batch = batch.to(device)

        if batch.x.shape[0] == 1:
            pass
        else:
            with torch.no_grad():
                pred = model(batch)

            y_true.append(batch.y.view(pred.shape).detach().cpu())
            y_pred.append(pred.detach().cpu())

    y_true = torch.cat(y_true, dim = 0).numpy()
    y_pred = torch.cat(y_pred, dim = 0).numpy()

    input_dict = {"y_true": y_true, "y_pred": y_pred}

    return evaluator.eval(input_dict)

In [25]:
model = GCN_Graph(args['hidden_dim'],
            dataset.num_tasks, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

In [26]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  print('Training...')
  loss = train(model, device, train_loader, optimizer, loss_fn)

  print('Evaluating...')
  train_result = eval(model, device, train_loader, evaluator)
  val_result = eval(model, device, valid_loader, evaluator)
  test_result = eval(model, device, test_loader, evaluator)

  train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Training...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

...

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 30, Loss: 0.0511, Train: 80.52%, Valid: 76.29% Test: 72.04%


In [27]:
train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator)[dataset.eval_metric]
test_acc = eval(best_model, device, test_loader, evaluator)[dataset.eval_metric]

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Best model: Train: 80.52%, Valid: 76.29% Test: 72.04%


## Faites des tests avec deux autres couches de global pooling de Pytorch Geometric.

In [28]:
# Réutilisez le code précédent (en changeant uniquement la couche de pooling)

### Global add pool

In [29]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool

### GCN to predict graph property
class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout):
        super(GCN_Graph, self).__init__()

        # Load encoders for Atoms in molecule graphs
        self.node_encoder = AtomEncoder(hidden_dim)

        # Node embedding model
        # Note that the input_dim and output_dim are set to hidden_dim
        self.gnn_node = GCN(9, hidden_dim,
            hidden_dim, num_layers, dropout, return_embeds=False)
        # Avant :
        # self.gnn_node = GCN(hidden_dim, hidden_dim,
        #   hidden_dim, num_layers, dropout, return_embeds=False)


        ############# Your code here ############
        ## Note:
        ## 1. Initialisez  self.pool par global mean pooling layer
        ## Lien vers la documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#global-pooling-layers
        ## (une ligne de code )

        #self.pool = None
        self.pool = global_add_pool
        #########################################

        # Output layer
        self.linear = torch.nn.Linear(hidden_dim, output_dim)


    def reset_parameters(self):
      self.gnn_node.reset_parameters()
      self.linear.reset_parameters()

    def forward(self, batched_data):
        # Input :  tensor batched_data,
        # Output : tensor for each graph.
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch
        embed = self.node_encoder(x)
        #out = None

        ############# Votre code ici ###########################################
        ## Note:
        ## 1. Contruire les plongements de noeuds en utilisant le GCN précédent
        ## 2. Utiliser global pooling layer pour constuire les features du graphe en entier
        ## Lien vers la documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#global-pooling-layers
        ## 3. Utilisez le couche linear layer  pour prédire la propriété du graphe

        out = self.gnn_node(x.float(), edge_index)
        out = self.pool(out, batch)  # Appliquer le pooling global
        out = self.linear(out)  # Couche de sortie linéaire

        ########################################################

        return out

In [30]:
model = GCN_Graph(args['hidden_dim'],
            dataset.num_tasks, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

In [31]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  print('Training...')
  loss = train(model, device, train_loader, optimizer, loss_fn)

  print('Evaluating...')
  train_result = eval(model, device, train_loader, evaluator)
  val_result = eval(model, device, valid_loader, evaluator)
  test_result = eval(model, device, test_loader, evaluator)

  train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Training...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

...

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 30, Loss: 0.8243, Train: 73.07%, Valid: 64.33% Test: 62.68%


In [32]:
train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator)[dataset.eval_metric]
test_acc = eval(best_model, device, test_loader, evaluator)[dataset.eval_metric]

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Best model: Train: 74.54%, Valid: 75.02% Test: 70.35%


### Global max pool

In [33]:
from ogb.graphproppred.mol_encoder import AtomEncoder
from torch_geometric.nn import global_add_pool, global_mean_pool, global_max_pool

### GCN to predict graph property
class GCN_Graph(torch.nn.Module):
    def __init__(self, hidden_dim, output_dim, num_layers, dropout):
        super(GCN_Graph, self).__init__()

        # Load encoders for Atoms in molecule graphs
        self.node_encoder = AtomEncoder(hidden_dim)

        # Node embedding model
        # Note that the input_dim and output_dim are set to hidden_dim
        self.gnn_node = GCN(9, hidden_dim,
            hidden_dim, num_layers, dropout, return_embeds=False)
        # Avant :
        # self.gnn_node = GCN(hidden_dim, hidden_dim,
        #   hidden_dim, num_layers, dropout, return_embeds=False)


        ############# Your code here ############
        ## Note:
        ## 1. Initialisez  self.pool par global mean pooling layer
        ## Lien vers la documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#global-pooling-layers
        ## (une ligne de code )

        #self.pool = None
        self.pool = global_max_pool
        #########################################

        # Output layer
        self.linear = torch.nn.Linear(hidden_dim, output_dim)


    def reset_parameters(self):
      self.gnn_node.reset_parameters()
      self.linear.reset_parameters()

    def forward(self, batched_data):
        # Input :  tensor batched_data,
        # Output : tensor for each graph.
        x, edge_index, batch = batched_data.x, batched_data.edge_index, batched_data.batch
        embed = self.node_encoder(x)
        #out = None

        ############# Votre code ici ###########################################
        ## Note:
        ## 1. Contruire les plongements de noeuds en utilisant le GCN précédent
        ## 2. Utiliser global pooling layer pour constuire les features du graphe en entier
        ## Lien vers la documentation:
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#global-pooling-layers
        ## 3. Utilisez le couche linear layer  pour prédire la propriété du graphe

        out = self.gnn_node(x.float(), edge_index)
        out = self.pool(out, batch)  # Appliquer le pooling global
        out = self.linear(out)  # Couche de sortie linéaire

        ########################################################

        return out

In [34]:
model = GCN_Graph(args['hidden_dim'],
            dataset.num_tasks, args['num_layers'],
            args['dropout']).to(device)
evaluator = Evaluator(name='ogbg-molhiv')

In [35]:
import copy

model.reset_parameters()

optimizer = torch.optim.Adam(model.parameters(), lr=args['lr'])
loss_fn = torch.nn.BCEWithLogitsLoss()

best_model = None
best_valid_acc = 0

for epoch in range(1, 1 + args["epochs"]):
  print('Training...')
  loss = train(model, device, train_loader, optimizer, loss_fn)

  print('Evaluating...')
  train_result = eval(model, device, train_loader, evaluator)
  val_result = eval(model, device, valid_loader, evaluator)
  test_result = eval(model, device, test_loader, evaluator)

  train_acc, valid_acc, test_acc = train_result[dataset.eval_metric], val_result[dataset.eval_metric], test_result[dataset.eval_metric]
  if valid_acc > best_valid_acc:
      best_valid_acc = valid_acc
      best_model = copy.deepcopy(model)
  print(f'Epoch: {epoch:02d}, '
        f'Loss: {loss:.4f}, '
        f'Train: {100 * train_acc:.2f}%, '
        f'Valid: {100 * valid_acc:.2f}% '
        f'Test: {100 * test_acc:.2f}%')

Training...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

...

Evaluating...


Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Epoch: 30, Loss: 0.0514, Train: 84.39%, Valid: 73.57% Test: 74.08%


In [36]:
train_acc = eval(best_model, device, train_loader, evaluator)[dataset.eval_metric]
valid_acc = eval(best_model, device, valid_loader, evaluator)[dataset.eval_metric]
test_acc = eval(best_model, device, test_loader, evaluator)[dataset.eval_metric]

print(f'Best model: '
      f'Train: {100 * train_acc:.2f}%, '
      f'Valid: {100 * valid_acc:.2f}% '
      f'Test: {100 * test_acc:.2f}%')

Iteration:   0%|          | 0/1029 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Iteration:   0%|          | 0/129 [00:00<?, ?it/s]

Best model: Train: 82.32%, Valid: 78.60% Test: 74.81%
